In [1]:
import json
import spacy
import nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.chunk.regexp import RegexpParser
#from allennlp.predictors.predictor import Predictor
#from allennlp_models import pretrained
nlp = spacy.load("en_core_web_sm")
#predictor = pretrained.load_predictor("structured-prediction-srl-bert")

In [2]:
#data = json.load(open("dev-SQuAD-v2.0.json"))

In [3]:
#data = data["data"]

In [4]:
import pandas as pd
data = pd.read_csv("20240402-train_modified_Antonym_allData_df.csv")

In [5]:
def hpsg(text):
    tokens = word_tokenize(text)
    
    # Perform part-of-speech tagging
    tagged_tokens = nltk.pos_tag(tokens)
    
    # Define a regular expression grammar for NP (noun phrase) chunks
    grammar = r"""
        NP: {<DT|JJ|NN.*>+} # Chunk sequences of DT, JJ, NN
    """
    
    # Create a chunk parser using the defined grammar
    chunk_parser = RegexpParser(grammar)
    
    # Parse the tagged tokens to identify NP chunks
    parsed_sentence = chunk_parser.parse(tagged_tokens)
    
    # Extract spans of NP chunks
    hpsg_list_doc = []
    for subtree in parsed_sentence.subtrees():
        if subtree.label() == 'NP':
            # Get the span of the NP chunk
            start = subtree.leaves()[0][1]
            end = subtree.leaves()[-1][1]
            hpsg_list_doc.append((start, end))
    hpsg_doc = []
    for unit in hpsg_list_doc:
        start = hpsg_list_doc.index(unit) + 1
        end = start
        span_str = f"({start}, {end})"
        hpsg_doc.append(span_str)
    return hpsg_doc

In [6]:
def generate_predicates(input_text):
    doc = nlp(input_text)
    pred_type_que = []
    pred_head_que = []

    # Iterate over each token in the document
    for token in doc:
        # Check if the token is a predicate (verb)
        if token.pos_ == "VERB":
            # Add the type of the predicate to pred_type_que
            pred_type_que.append(token.dep_)
            # Add the index of the head of the predicate to pred_head_que
            pred_head_que.append(token.head.i + 1)  # Adding 1 to convert to 1-indexing

    return pred_type_que, pred_head_que

In [7]:
def convert_input_to_output(context, question_data, id):
    output = {
        "tag_rep": {
            "hpsg_list_doc": [],
            "srlspan_pred_doc": [],
            "pred_type_que": [],
            "pred_head_que": [],
            "srlspan_pred_que": [],
            "hpsg_list_que": [],
            "pred_type_doc": [],
            "srldep_pred_doc": [],
            "que_tokens": question_data.split(),
            "doc_tokens": [sentence.split() for sentence in context.split('. ')],
            "pred_head_doc": [],
            "srldep_pred_que": []
        },
        "qas_id": id
    }
    
    doc_sentences = [sentence.strip() for sentence in context.split('.')]
    doc_tokens = [[token.text for token in nlp(sentence)] for sentence in doc_sentences]

    # Tokenize the sentence
    
        
    # Populate the output dictionary with the extracted information
    output["tag_rep"]["hpsg_list_doc"] = hpsg(context)
    output["tag_rep"]["hpsg_list_que"] = hpsg(question_data)
    output["tag_rep"]["pred_type_doc"], output["tag_rep"]["pred_head_doc"] = generate_predicates(context)
    output["tag_rep"]["pred_type_que"], output["tag_rep"]["pred_head_que"] = generate_predicates(question_data)
    return output



In [8]:
string = ""
counter = 0
from tqdm import tqdm
for index, row in tqdm(data.iterrows(),total=data.shape[0]):
    curstring = json.dumps(convert_input_to_output(row["context"], row["modified question"], row["qid"]))
    string += str(curstring) + "\n"

100%|███████████████████████████████████████████████████████████████████████| 324814/324814 [11:42:06<00:00,  7.71it/s]


In [9]:
#import re

# Assuming 'string' contains JSON data with single quotes
# Use regular expressions to replace single quotes outside of string values with double quotes
#string = re.sub(r"(?<!\\)'(?!s\b)", '"', string)
with open("antonym_train_span.json", "w", encoding="utf-8") as file:
    file.write(string)